In [1]:
import requests
import numpy as np
import pandas as pd

In [28]:
df1 = pd.read_csv(r"https://raw.githubusercontent.com/Perovsky56/BankLoanModel/refs/heads/main/datasets/Loan.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [30]:
print(df1.columns)

Index(['ApplicationDate', 'Age', 'AnnualIncome', 'CreditScore', 'EmploymentStatus',
       'EducationLevel', 'Experience', 'LoanAmount', 'LoanDuration', 'MaritalStatus',
       'NumberOfDependents', 'HomeOwnershipStatus', 'MonthlyDebtPayments',
       'CreditCardUtilizationRate', 'NumberOfOpenCreditLines', 'NumberOfCreditInquiries',
       'DebtToIncomeRatio', 'BankruptcyHistory', 'LoanPurpose', 'PreviousLoanDefaults',
       'PaymentHistory', 'LengthOfCreditHistory', 'SavingsAccountBalance',
       'CheckingAccountBalance', 'TotalAssets', 'TotalLiabilities', 'MonthlyIncome',
       'UtilityBillsPaymentHistory', 'JobTenure', 'NetWorth', 'BaseInterestRate', 'InterestRate',
       'MonthlyLoanPayment', 'TotalDebtToIncomeRatio', 'LoanApproved', 'RiskScore'],
      dtype='object')


In [31]:
print(df1.head(3))

  ApplicationDate  Age  AnnualIncome  CreditScore EmploymentStatus EducationLevel  Experience  \
0      2018-01-01   45         39948          617         Employed         Master          22   
1      2018-01-02   38         39709          628         Employed      Associate          15   
2      2018-01-03   47         40724          570         Employed       Bachelor          26   

   LoanAmount  LoanDuration MaritalStatus  NumberOfDependents HomeOwnershipStatus  \
0       13152            48       Married                   2                 Own   
1       26045            48        Single                   1            Mortgage   
2       17627            36       Married                   2                Rent   

   MonthlyDebtPayments  CreditCardUtilizationRate  NumberOfOpenCreditLines  \
0                  183                   0.354418                        1   
1                  496                   0.087827                        5   
2                  902              

In [33]:
df1 = df1.drop(['ApplicationDate', 'CreditScore', 'HomeOwnershipStatus', 'CreditCardUtilizationRate', 'NumberOfOpenCreditLines', 'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'LoanPurpose', 'PreviousLoanDefaults', 'PaymentHistory', 'LengthOfCreditHistory', 'UtilityBillsPaymentHistory', 'BaseInterestRate', 'InterestRate', 'TotalDebtToIncomeRatio', 'LoanApproved', 'RiskScore'], axis=1)

In [35]:
df1.head(3)

,Age,AnnualIncome,EmploymentStatus,EducationLevel,Experience,LoanAmount,LoanDuration,MaritalStatus,NumberOfDependents,MonthlyDebtPayments,BankruptcyHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,JobTenure,NetWorth,MonthlyLoanPayment
0,45,39948,Employed,Master,22,13152,48,Married,2,183,0,7632,1202,146111,19183,3329.000000,11,126928,419.805992
1,38,39709,Employed,Associate,15,26045,48,Single,1,496,0,4627,3460,53204,9595,3309.083333,3,43609,794.054238
2,47,40724,Employed,Bachelor,26,17627,36,Married,2,902,0,886,895,25176,128874,3393.666667,6,5205,666.406688


In [39]:
columns_to_move = ['LoanAmount', 'LoanDuration']

cols = [col for col in df1.columns if col not in columns_to_move] + columns_to_move
df1 = df1[cols]

In [41]:
df1.sample(15)

,Age,AnnualIncome,EmploymentStatus,EducationLevel,Experience,MaritalStatus,NumberOfDependents,MonthlyDebtPayments,BankruptcyHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,JobTenure,NetWorth,MonthlyLoanPayment,LoanAmount,LoanDuration
12101,28,72217,Employed,Associate,0,Married,1,500,0,2752,259,48301,18846,6018.083333,6,29455,968.211564,28340,48
5216,43,33601,Employed,Master,19,Married,2,430,0,1666,6517,49606,47704,2800.083333,4,1902,627.983944,21981,48
7548,48,15000,Employed,Bachelor,24,Married,3,642,0,5394,767,142196,112721,1250.000000,4,29475,1668.852587,51554,60
5803,61,65333,Employed,High School,37,Single,2,447,0,526,539,20284,75656,5444.416667,5,6150,1069.290416,20604,24
14633,33,72244,Employed,Associate,10,Married,2,499,0,1374,398,19076,26769,6020.333333,7,10058,1551.574746,16125,12
4808,57,48416,Employed,Bachelor,32,Divorced,2,669,0,8695,1222,40899,55465,4034.666667,7,6170,418.845570,15923,72
5831,38,28778,Employed,Bachelor,12,Married,2,464,0,1552,1121,55094,11224,2398.166667,3,43870,663.543254,23911,60
12239,35,38681,Employed,Master,13,Married,1,675,1,1737,225,51627,16100,3223.416667,5,35527,515.836702,15515,48
19415,43,42374,Employed,High School,18,Married,2,689,0,1124,349,21490,156572,3531.166667,10,3769,722.357710,21546,48
16650,53,26122,Employed,Associate,28,Divorced,1,200,0,1135,825,49538,83726,2176.833333,5,2117,987.484931,27362,36


In [42]:
df1.shape

(20000, 19)

In [43]:
df1.dtypes

,0
Age,int64
AnnualIncome,int64
EmploymentStatus,object
EducationLevel,object
Experience,int64
MaritalStatus,object
NumberOfDependents,int64
MonthlyDebtPayments,int64
BankruptcyHistory,int64
SavingsAccountBalance,int64


In [44]:
df1.describe()

,Age,AnnualIncome,Experience,NumberOfDependents,MonthlyDebtPayments,BankruptcyHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,JobTenure,NetWorth,MonthlyLoanPayment,LoanAmount,LoanDuration
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04,2.000000e+04,20000.000000,20000.000000,2.000000e+04,20000.000000,20000.000000,20000.000000
mean,39.752600,59161.473550,17.522750,1.517300,454.292700,0.052400,4946.052150,1782.555100,9.696440e+04,3.625241e+04,4891.715521,5.002650,7.229432e+04,911.607052,24882.867800,54.057000
std,11.622713,40350.845168,11.316836,1.386325,240.507609,0.222838,6604.887477,2245.378812,1.207999e+05,4.725151e+04,3296.771598,2.236804,1.179200e+05,674.583473,13427.421217,24.664857
min,18.000000,15000.000000,0.000000,0.000000,50.000000,0.000000,73.000000,24.000000,2.098000e+03,3.720000e+02,1250.000000,0.000000,1.000000e+03,97.030193,3674.000000,12.000000
25%,32.000000,31679.000000,9.000000,0.000000,286.000000,0.000000,1541.750000,551.000000,3.118025e+04,1.119675e+04,2629.583333,3.000000,8.734750e+03,493.763700,15575.000000,36.000000
50%,40.000000,48566.000000,17.000000,1.000000,402.000000,0.000000,2986.000000,1116.000000,6.069900e+04,2.220300e+04,4034.750000,5.000000,3.285550e+04,728.511452,21914.500000,48.000000
75%,48.000000,74391.000000,25.000000,2.000000,564.000000,0.000000,5873.250000,2126.000000,1.174052e+05,4.314650e+04,6163.000000,6.000000,8.882550e+04,1112.770759,30835.000000,72.000000
max,80.000000,485341.000000,61.000000,5.000000,2919.000000,1.000000,200089.000000,52572.000000,2.619627e+06,1.417302e+06,25000.000000,16.000000,2.603208e+06,10892.629520,184732.000000,120.000000


In [45]:
nulls_summary = pd.DataFrame(df1.isnull().any(), columns=['Nulls'])
nulls_summary['Num_of_nulls [qty]'] = pd.DataFrame(df1.isnull().sum())
nulls_summary['Num_of_nulls [%]'] = round((df1.isnull().mean()*100),2)
print(nulls_summary)

                        Nulls  Num_of_nulls [qty]  Num_of_nulls [%]
Age                     False                   0               0.0
AnnualIncome            False                   0               0.0
EmploymentStatus        False                   0               0.0
EducationLevel          False                   0               0.0
Experience              False                   0               0.0
MaritalStatus           False                   0               0.0
NumberOfDependents      False                   0               0.0
MonthlyDebtPayments     False                   0               0.0
BankruptcyHistory       False                   0               0.0
SavingsAccountBalance   False                   0               0.0
CheckingAccountBalance  False                   0               0.0
TotalAssets             False                   0               0.0
TotalLiabilities        False                   0               0.0
MonthlyIncome           False                   